In [3]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.fftpack import fft
from tkinter import TclError
import pandas as pd
from datetime import datetime

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second
THRESHOLD_X = 2000
THRESHOLD_Y = 0.6
FILENAME = "output.csv"

In [4]:
# Creating dataframe for frequency
threshold_df = pd.DataFrame(columns=['Frequency', 'Loudness', 'TimeStamp'])

# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# stream object to get data from microphone
stream = pyaudio.PyAudio().open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', color="blue", lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', color="blue", lw=2)

# format waveform axes
ax1.set_title('Audio Visualizer')
ax1.set_xlabel('Chunks')
ax1.set_ylabel('Loudness')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])


ax2.set_xlabel('Frequency (Log10)')
ax2.set_ylabel('Loudness')
plt.axhspan(0, THRESHOLD_Y, 0, 0.658, facecolor='green', alpha=0.7)
plt.axhspan(THRESHOLD_Y, 1.0, 0.658, RATE / 2, facecolor='red', alpha=0.7)
plt.axhspan(0, THRESHOLD_Y, 0.658, RATE / 2, facecolor='orange', alpha=0.7)
plt.axhspan(THRESHOLD_Y, 1.0, 0, 0.658, facecolor='yellow', alpha=0.7)

pop_g = mpatches.Patch(color='green', label='NORMAL Noise Levels')
pop_o = mpatches.Patch(color='orange', label='MODERATE Noise Levels')
pop_r = mpatches.Patch(color='red', label='SEVERE Noise Levels')
pop_y = mpatches.Patch(color='yellow', label='SLIGHT Noise Levels')
plt.legend(handles=[pop_g, pop_o, pop_r, pop_y], ncol=4, loc='upper left', fancybox=True, shadow=True)

# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')

df_length = 0

while True:
    
    # binary data
    data = stream.read(CHUNK, exception_on_overflow = False)  
    
    print(len(data))
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128

    line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_int)
    temp = np.abs(yf[0:CHUNK]) / (128 * CHUNK)
    line_fft.set_ydata(temp)
    (x_line_fft, y_line_fft) = line_fft.get_data()
    x_line_fft = np.array(x_line_fft)[10:]
    y_line_fft = np.array(y_line_fft)[10:]
    threshold_arr = np.where(np.logical_and(x_line_fft > THRESHOLD_X, y_line_fft > THRESHOLD_Y))
    if (len(threshold_arr[0]) > 0 and x_line_fft[threshold_arr[0][0]] < RATE / 2):
        threshold_df.loc[df_length] = [x_line_fft[threshold_arr[0][0]], y_line_fft[threshold_arr[0][0]], datetime.now().strftime("%d/%m/%Y %H:%M:%S")]
        df_length += 1
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        
    except TclError:
        print('stream stopped')
        break

threshold_df.to_csv(FILENAME, mode='w')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/matplotlib/scale.py:253: RuntimeWarning: overflow encountered in power
  return np.power(self.base, a)


stream started
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096


KeyboardInterrupt: 

: 